In [1]:
import unittest
import os,sys
os.environ["MKL_NUM_THREADS"] = "4" # export MKL_NUM_THREADS=6
import numpy as np
import logging
%load_ext line_profiler
import dask.array as da
os.chdir('../')
sys.path.append('main/')
from full_gradient import *
os.getcwd()

'/home/rrobin/Documents/Stellacage_code'

In [2]:
print(os.getcwd())
full_grad=Full_gradient('config_file/config.ini')
def f(param_array):
    (m,n,Rmn,Zmn)=full_grad.S_parametrization
    R=param_array[:len(m)]
    Z=param_array[len(m):]
    param=((m,n,R,Z))
    cost=full_grad.cost(param)
    print('temporary costs : \n Regcoil {:5e} \n distance {:5e} \n perimeter {:5e}'.format(cost[0],cost[1],cost[2]))
    return cost.sum()
def gradf(param_array):
    (m,n,Rmn,Zmn)=full_grad.S_parametrization
    R=param_array[:len(m)]
    Z=param_array[len(m):]
    param=((m,n,R,Z))
    return (full_grad.grad_cost(param)).sum(axis=0)

/home/rrobin/Documents/Stellacage_code


In [3]:
eps=1e-9
S_param_complete=full_grad.S_parametrization
ls=len(S_param_complete[0])
        
#we apply one pertubation
perturb=(2*np.random.random(2*ls)-1)
S_param=np.concatenate((S_param_complete[2],S_param_complete[3]))
new_param=S_param+eps*perturb
t0=time.time()
old_cost=f(S_param)
new_cost=f(new_param)
gradient=gradf(S_param)
print('time used : {}s'.format(time.time()-t0))

temporary costs : 
 Regcoil 1.336885e+00 
 distance 1.963323e-01 
 perimeter 4.922275e-01
temporary costs : 
 Regcoil 1.336885e+00 
 distance 1.963322e-01 
 perimeter 4.922275e-01
time used : 28.574032306671143s


In [4]:
print('numerical cost obtained: {:10e} and {:10e}'.format(old_cost,new_cost))
print('numerical gradient : {:10e}'.format((new_cost-old_cost)/eps))
print('gradient computed : {:10e}'.format(np.dot(gradient,perturb)))

numerical cost obtained: 2.025445e+00 and 2.025444e+00
numerical gradient : -6.945399e+01
gradient computed : -6.945442e+01


In [5]:
import scipy.optimize
optimize_shape=scipy.optimize.minimize(f, S_param, method='CG', jac=gradf,options={'maxiter':100,'return_all':True})

temporary costs : 
 Regcoil 1.336885e+00 
 distance 1.963323e-01 
 perimeter 4.922275e-01
temporary costs : 
 Regcoil 2.340321e+00 
 distance   inf 
 perimeter   inf
main/full_gradient.py:56: RuntimeWarning: invalid value encountered in multiply
  X=-1*vgradf[:,:,np.newaxis]*grad_d_min
main/full_gradient.py:108: RuntimeWarning: invalid value encountered in multiply
  return grad_f_e(perim)*grad_perim
temporary costs : 
 Regcoil 1.336885e+00 
 distance 1.963323e-01 
 perimeter 4.922275e-01
temporary costs : 
 Regcoil 2.340321e+00 
 distance   inf 
 perimeter   inf
temporary costs : 
 Regcoil 1.818106e+00 
 distance   inf 
 perimeter   inf
temporary costs : 
 Regcoil 1.522346e+00 
 distance   inf 
 perimeter 6.958816e-02
temporary costs : 
 Regcoil 1.476999e+00 
 distance 3.274445e-01 
 perimeter 0.000000e+00
temporary costs : 
 Regcoil 1.477983e+00 
 distance 1.592468e-01 
 perimeter 0.000000e+00
temporary costs : 
 Regcoil 1.477184e+00 
 distance 1.770147e-02 
 perimeter 0.000000e+00
t

In [6]:
import pickle
with open(full_grad.config['other']['path_output'],'wb') as output_file:
    pickle.dump(optimize_shape,output_file)

In [7]:
optimize_shape2=scipy.optimize.minimize(f, optimize_shape.x, method='CG', jac=gradf,options={'maxiter':100,'return_all':True})

temporary costs : 
 Regcoil 9.020159e-01 
 distance 1.940695e-03 
 perimeter 0.000000e+00
temporary costs : 
 Regcoil 2.385042e+00 
 distance   inf 
 perimeter   inf
main/full_gradient.py:56: RuntimeWarning: invalid value encountered in multiply
  X=-1*vgradf[:,:,np.newaxis]*grad_d_min
main/full_gradient.py:108: RuntimeWarning: invalid value encountered in multiply
  return grad_f_e(perim)*grad_perim
temporary costs : 
 Regcoil 9.020159e-01 
 distance 1.940695e-03 
 perimeter 0.000000e+00
temporary costs : 
 Regcoil 2.385042e+00 
 distance   inf 
 perimeter   inf
temporary costs : 
 Regcoil 1.821973e+00 
 distance   inf 
 perimeter   inf
temporary costs : 
 Regcoil 1.394647e+00 
 distance   inf 
 perimeter   inf
temporary costs : 
 Regcoil 9.103222e-01 
 distance   inf 
 perimeter   inf
temporary costs : 
 Regcoil 8.602175e-01 
 distance   inf 
 perimeter   inf
temporary costs : 
 Regcoil 8.260542e-01 
 distance   inf 
 perimeter 1.762486e+02
temporary costs : 
 Regcoil 8.561876e-01 
 

In [8]:
optimize_shape2

, -4.44209242e-03, -9.09378568e-03,
        4.43132379e-03,  2.92608384e-03, -3.93250179e-03,  3.00321819e-03,
        1.27330374e-02,  4.65397360e-03,  5.02558028e-03,  6.49201786e-03,
        1.13595748e-02,  8.09958102e-03, -1.05280464e-02, -2.76474076e-02,
        5.43312552e-04,  1.27070622e-03, -9.19776287e-04, -1.24948111e-03,
        1.48277873e-03, -5.07754542e-03,  1.11392206e-03, -3.01654234e-03,
        2.87208895e-03, -1.77054008e-03, -1.44284839e-04,  4.55909777e-04,
        2.32792762e-03, -7.84282926e-03, -4.68684678e-03, -2.09822842e-03,
       -2.79596395e-02, -4.63082942e-03,  1.42103728e-02,  3.11089992e-02,
        1.02343305e-02,  1.42786817e-03, -1.00550929e-02, -2.42364939e-03,
       -3.82764567e-03,  3.00158705e-03,  4.85544530e-03, -3.32408026e-03,
        2.42181863e-03, -3.40884748e-03, -1.29433343e-03, -6.83156362e-04,
        4.26554236e-03,  1.04360798e-02,  3.45197428e-04,  9.40787263e-04,
        1.35307901e-02,  1.39175725e-02, -9.57492879e-03, -1.532

In [4]:
%lprun -u 1 -f full_grad.grad_cost full_grad.grad_cost(full_grad.S_parametrization)

Timer unit: 1 s

Total time: 20.9241 s
File: main/full_gradient.py
Function: grad_cost at line 84

Line #      Hits         Time  Per Hit   % Time  Line Contents
    84                                               def grad_cost(self,paramS):
    85         1          0.2      0.2      1.1          S=Toroidal_surface(paramS,(self.ntheta_coil,self.nzeta_coil),self.Np)
    86         1         17.3     17.3     82.7          result=self.compute_gradient_of(S=S)
    87         1          0.0      0.0      0.0          I_vector,I_matrix=result['I1']
    88         1          0.6      0.6      2.7          X,Y=self.gradient_cost_distance(result['T'],S)
    89         1          2.8      2.8     13.1          theta,dtildetheta,dtheta,dSdtheta=S.get_theta_pertubation()
    90         1          0.1      0.1      0.3          grad=np.einsum('ija,oija,ij->o',I_vector,theta,S.dS/S.npts)+np.einsum('ijab,oijab,ij->o',I_matrix,dtildetheta,S.dS/S.npts)
    91         1          0.0      0.0      0.1

In [ ]:
print(optimize_shape)